In [ ]:
import SimpleITK as sitk
import numpy as np
import os

def load_image(image_path):
    """Loads a NIfTI medical image using SimpleITK."""
    return sitk.ReadImage(image_path)

def get_bounding_box(mask, margin=5):
    """
    Computes the bounding box of a binary mask and removes empty slices.

    Args:
        mask: SimpleITK Image object representing the segmentation mask.
        margin: Extra padding to ensure structures aren't cut off.

    Returns:
        Cropping coordinates (crop_x, crop_y, crop_z) as tuples.
    """
    mask_array = sitk.GetArrayFromImage(mask)  # Convert mask to numpy array
    indices = np.where(mask_array > 0)  # Get all nonzero voxel indices

    if len(indices[0]) == 0:
        raise ValueError("Mask is empty, cannot compute bounding box.")

    # Compute bounding box
    z_min, z_max = np.min(indices[0]), np.max(indices[0])
    y_min, y_max = np.min(indices[1]), np.max(indices[1])
    x_min, x_max = np.min(indices[2]), np.max(indices[2])

    # Add margin while ensuring bounds are not exceeded
    x_min = max(0, x_min - margin)
    x_max = min(mask.GetSize()[0], x_max + margin)
    y_min = max(0, y_min - margin)
    y_max = min(mask.GetSize()[1], y_max + margin)
    z_min = max(0, z_min - margin)
    z_max = min(mask.GetSize()[2], z_max + margin)

    return (x_min, x_max), (y_min, y_max), (z_min, z_max)

def crop_image(image, crop_x, crop_y, crop_z):
    """
    Crops the image along the x, y, and z axes based on the given ranges.

    Args:
        image: SimpleITK Image object.
        crop_x: Tuple of (start_x, end_x) for cropping along the x-axis.
        crop_y: Tuple of (start_y, end_y) for cropping along the y-axis.
        crop_z: Tuple of (start_z, end_z) for cropping along the z-axis.

    Returns:
        Cropped SimpleITK Image object.
    """
    # Ensure integer values for SimpleITK
    start_index = (int(crop_x[0]), int(crop_y[0]), int(crop_z[0]))
    cropped_size = [
        int(crop_x[1] - crop_x[0]),  # X cropping
        int(crop_y[1] - crop_y[0]),  # Y cropping
        int(crop_z[1] - crop_z[0]),  # Z cropping
    ]

    cropped_image = sitk.RegionOfInterest(image, size=cropped_size, index=start_index)
    return cropped_image

def process_dataset(data_folder, output_folder):
    """
    Processes all NIfTI images in the dataset, removing empty slices and cropping to the mask region.

    Args:
        data_folder: Folder containing NIfTI images and masks.
        output_folder: Folder to save cropped images.
    """
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(data_folder):
        if filename.endswith(".nii.gz") and "label" not in filename:  # Process only image files
            image_path = os.path.join(data_folder, filename)
            label_path = image_path.replace(".img.nii.gz", ".label.nii.gz")  # Assuming label has "label" in the name

            if not os.path.exists(label_path):
                print(f"Skipping {filename} (No corresponding mask found)")
                continue

            # Load image and label
            image = load_image(image_path)
            label = load_image(label_path)

            # Compute cropping bounds
            crop_x, crop_y, crop_z = get_bounding_box(label, margin=5)

            # Crop images and labels
            cropped_image = crop_image(image, crop_x, crop_y, crop_z)
            cropped_label = crop_image(label, crop_x, crop_y, crop_z)

            # Save cropped images
            output_image_path = os.path.join(output_folder, f"cropped_{filename}.nii.gz")
            output_label_path = os.path.join(output_folder, f"cropped_{filename.replace('.img', '.label')}")
            
            sitk.WriteImage(cropped_image, output_image_path)
            sitk.WriteImage(cropped_label, output_label_path)

            print(f"Processed {filename}: Cropped to X{crop_x}, Y{crop_y}, Z{crop_z}")

# Select folders here
data_folder = "D:\\Electra\\data\\AllData"
output_folder = "D:\\Electra\\data\\Cropped_AllData"
process_dataset(data_folder, output_folder)

Processed 1.img.nii.gz: Cropped to X(100, 417), Y(182, 465), Z(12, 187)
Processed 10.img.nii.gz: Cropped to X(82, 411), Y(179, 450), Z(36, 227)
Processed 100.img.nii.gz: Cropped to X(109, 449), Y(112, 464), Z(0, 180)
Processed 1000.img.nii.gz: Cropped to X(90, 435), Y(143, 447), Z(40, 219)
Processed 101.img.nii.gz: Cropped to X(146, 436), Y(106, 418), Z(13, 185)
Processed 102.img.nii.gz: Cropped to X(138, 412), Y(149, 432), Z(73, 248)
Processed 103.img.nii.gz: Cropped to X(126, 466), Y(88, 360), Z(30, 197)
Processed 104.img.nii.gz: Cropped to X(121, 415), Y(160, 433), Z(54, 216)
Processed 105.img.nii.gz: Cropped to X(97, 465), Y(147, 425), Z(56, 233)
Processed 106.img.nii.gz: Cropped to X(87, 383), Y(165, 429), Z(0, 193)
Processed 107.img.nii.gz: Cropped to X(85, 415), Y(135, 426), Z(31, 221)
Processed 108.img.nii.gz: Cropped to X(90, 417), Y(221, 486), Z(44, 226)
Processed 109.img.nii.gz: Cropped to X(109, 424), Y(159, 397), Z(31, 230)
Processed 11.img.nii.gz: Cropped to X(104, 416), 

In [8]:
import os
import shutil
from glob import glob

# Paths
source_folder = "D:\\Electra\\data\\Cropped_AllData"
train_folder = "D:\\Electra\\data\\Cropped_TrainingData"
val_folder   = "D:\\Electra\\data\\Cropped_ValidationData"
test_folder  = "D:\\Electra\\data\\Cropped_TestData"

# Create target directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all images sorted by index
images = sorted(glob(os.path.join(source_folder, "cropped_*.img.nii.gz")))
labels = sorted(glob(os.path.join(source_folder, "cropped_*.label.nii.gz")))

assert len(images) == len(labels), "Mismatch between images and labels"

# Move files based on index, change if you want different train/val/test splits
for i, (img, lbl) in enumerate(zip(images, labels)):
    if i <= 640:
        out_folder = train_folder
    elif 641 <= i <= 800:
        out_folder = val_folder
    else:
        out_folder = test_folder

    shutil.copy(img, os.path.join(out_folder, os.path.basename(img)))
    shutil.copy(lbl, os.path.join(out_folder, os.path.basename(lbl)))

print(f"Done! Split {len(images)} files into train/val/test folders.")

Done! Split 1000 files into train/val/test folders.


In [ ]:
# Do the rest only if you have a lot of memory & a powerful GPU

def resize_to_shape_itk(image, output_shape=(128, 128, 128), is_label=False):
    original_size = image.GetSize()
    original_spacing = image.GetSpacing()

    new_spacing = [
        orig_sz * orig_spc / new_sz
        for orig_sz, orig_spc, new_sz in zip(original_size, original_spacing, output_shape)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(output_shape)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resampler.SetInterpolator(sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear)

    return resampler.Execute(image)

def resize_dataset(input_folder, output_folder, output_shape=(128, 128, 128)):
    os.makedirs(output_folder, exist_ok=True)

    all_files = sorted(f for f in os.listdir(input_folder) if f.endswith(".img.nii.gz"))
    index = 1

    for img_filename in all_files:
        label_filename = img_filename.replace(".img.nii.gz", ".label.nii.gz")

        img_path = os.path.join(input_folder, img_filename)
        label_path = os.path.join(input_folder, label_filename)

        if not os.path.exists(label_path):
            print(f"Skipping {img_filename} (label not found)")
            continue

        print(f"Resizing {img_filename} and {label_filename} to {output_shape}...")

        # Read and resize
        image = sitk.ReadImage(img_path)
        label = sitk.ReadImage(label_path)

        resized_image = resize_to_shape_itk(image, output_shape)
        resized_label = resize_to_shape_itk(label, output_shape, is_label=True)

        # Trick: Save with .nii.gz, then rename to .img.nii.gz
        real_image_out = os.path.join(output_folder, f"resized_{index}.nii.gz")
        final_image_out = os.path.join(output_folder, f"resized_{index}.img.nii.gz")

        real_label_out = os.path.join(output_folder, f"resized_{index}.label.nii.gz")

        # Save as .nii.gz
        sitk.WriteImage(resized_image, real_image_out, useCompression=True)
        sitk.WriteImage(resized_label, real_label_out, useCompression=True)

        # Rename the image to .img.nii.gz (leave label as is)
        os.rename(real_image_out, final_image_out)

        index += 1

In [5]:
# Run the code
input_folder = "D:\\Electra\\data\\Cropped_AllData"
output_folder = "D:\\Electra\\data\\Resized_AllData"

resize_dataset(input_folder, output_folder)

Resizing cropped_1.img.nii.gz and cropped_1.label.nii.gz to (128, 128, 128)...
Resizing cropped_10.img.nii.gz and cropped_10.label.nii.gz to (128, 128, 128)...
Resizing cropped_100.img.nii.gz and cropped_100.label.nii.gz to (128, 128, 128)...
Resizing cropped_1000.img.nii.gz and cropped_1000.label.nii.gz to (128, 128, 128)...
Resizing cropped_101.img.nii.gz and cropped_101.label.nii.gz to (128, 128, 128)...
Resizing cropped_102.img.nii.gz and cropped_102.label.nii.gz to (128, 128, 128)...
Resizing cropped_103.img.nii.gz and cropped_103.label.nii.gz to (128, 128, 128)...
Resizing cropped_104.img.nii.gz and cropped_104.label.nii.gz to (128, 128, 128)...
Resizing cropped_105.img.nii.gz and cropped_105.label.nii.gz to (128, 128, 128)...
Resizing cropped_106.img.nii.gz and cropped_106.label.nii.gz to (128, 128, 128)...
Resizing cropped_107.img.nii.gz and cropped_107.label.nii.gz to (128, 128, 128)...
Resizing cropped_108.img.nii.gz and cropped_108.label.nii.gz to (128, 128, 128)...
Resizing

In [7]:
import os
import shutil
from glob import glob

# Paths
source_folder = "D:\\Electra\\data\\Resized_AllData"
train_folder = "D:\\Electra\\data\\Resized_TrainingData"
val_folder   = "D:\\Electra\\data\\Resized_ValidationData"
test_folder  = "D:\\Electra\\data\\Resized_TestData"

# Create target directories
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all images sorted by index
images = sorted(glob(os.path.join(source_folder, "resized_*.img.nii.gz")))
labels = sorted(glob(os.path.join(source_folder, "resized_*.label.nii.gz")))

assert len(images) == len(labels), "Mismatch between images and labels"

# Move files based on index, change if you want different train/val/test splits
for i, (img, lbl) in enumerate(zip(images, labels)):
    if i <= 640:
        out_folder = train_folder
    elif 641 <= i <= 800:
        out_folder = val_folder
    else:
        out_folder = test_folder

    shutil.copy(img, os.path.join(out_folder, os.path.basename(img)))
    shutil.copy(lbl, os.path.join(out_folder, os.path.basename(lbl)))

print(f"Done! Split {len(images)} files into train/val/test folders.")

Done! Split 1000 files into train/val/test folders.


In [11]:
import os
import SimpleITK as sitk

data_folder = "D:\\Electra\\data\\Cropped_AllData" 
orientations = {}

for filename in sorted(os.listdir(data_folder)):
    if filename.endswith(".img.nii.gz") and "label" not in filename:
        image_path = os.path.join(data_folder, filename)
        image = sitk.ReadImage(image_path)
        direction = image.GetDirection()  # A tuple of 9 numbers (3x3 matrix)

        # Track orientation as string
        if direction not in orientations:
            orientations[direction] = []
        orientations[direction].append(filename)

# Print results
print(f"\n Found {len(orientations)} unique orientations:\n")
for i, (dir_matrix, files) in enumerate(orientations.items()):
    print(f"Orientation {i+1}: {dir_matrix}")
    print(f" - {len(files)} files: {files[:5]}{' ...' if len(files) > 5 else ''}\n")



 Found 1 unique orientations:

Orientation 1: (1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)
 - 1000 files: ['cropped_1.img.nii.gz', 'cropped_10.img.nii.gz', 'cropped_100.img.nii.gz', 'cropped_1000.img.nii.gz', 'cropped_101.img.nii.gz'] ...



In [1]:
import nibabel as nib
import numpy as np
import os

# Specify the directory containing your .nii.gz images
image_folder = "D:\\Electra\\data\\Cropped_AllData"  # Replace with your folder path
image_files = [f for f in os.listdir(image_folder) if f.endswith('img.nii.gz')]

# Check if we have at least 1000 images
if len(image_files) < 1000:
    print(f"Warning: Only {len(image_files)} images found. Proceeding with available images.")
else:
    image_files = image_files[:1000]  # Limit to 1000 images if more are present

# List to store mean intensities
mean_intensities = []

# Process each .nii.gz file
for image_file in image_files:
    # Load the NIfTI image
    img_path = os.path.join(image_folder, image_file)
    try:
        nii_img = nib.load(img_path)
        img_data = nii_img.get_fdata()  # Get the image data as a NumPy array
        
        # Calculate mean intensity of the image
        mean_intensity = np.mean(img_data)
        mean_intensities.append(mean_intensity)
        
    except Exception as e:
        print(f"Failed to process {image_file}: {str(e)}")
        continue

# Calculate overall mean intensity across all images
if mean_intensities:
    overall_mean = np.mean(mean_intensities)
    print(f"Mean intensity across {len(mean_intensities)} images: {overall_mean:.2f}")
else:
    print("No images were processed successfully.")

# Optional: Print mean intensity for each image
for i, (file, intensity) in enumerate(zip(image_files, mean_intensities)):
    print(f"Image {i+1}: {file} - Mean Intensity: {intensity:.2f}")

Mean intensity across 1000 images: 41.74
Image 1: cropped_1.img.nii.gz - Mean Intensity: 28.18
Image 2: cropped_10.img.nii.gz - Mean Intensity: 20.72
Image 3: cropped_100.img.nii.gz - Mean Intensity: -10.66
Image 4: cropped_1000.img.nii.gz - Mean Intensity: 12.43
Image 5: cropped_101.img.nii.gz - Mean Intensity: 65.82
Image 6: cropped_102.img.nii.gz - Mean Intensity: 1.42
Image 7: cropped_103.img.nii.gz - Mean Intensity: 67.29
Image 8: cropped_104.img.nii.gz - Mean Intensity: -0.75
Image 9: cropped_105.img.nii.gz - Mean Intensity: 81.72
Image 10: cropped_106.img.nii.gz - Mean Intensity: -7.68
Image 11: cropped_107.img.nii.gz - Mean Intensity: 4.78
Image 12: cropped_108.img.nii.gz - Mean Intensity: 35.70
Image 13: cropped_109.img.nii.gz - Mean Intensity: 55.19
Image 14: cropped_11.img.nii.gz - Mean Intensity: 12.61
Image 15: cropped_110.img.nii.gz - Mean Intensity: 38.69
Image 16: cropped_111.img.nii.gz - Mean Intensity: -27.22
Image 17: cropped_112.img.nii.gz - Mean Intensity: 66.91
Im

In [1]:
import os
import nibabel as nib
import numpy as np

folder = "D:/Electra/data/Cropped_AllData"

all_means = []
all_mins = []
all_maxs = []

for fname in os.listdir(folder):
    if fname.endswith(".nii.gz") and "label" not in fname:  # Αγνόησε τα labels
        img_path = os.path.join(folder, fname)
        img = nib.load(img_path)
        data = img.get_fdata()

        data_nonzero = data[data > 0]

        mean_val = np.mean(data_nonzero)
        min_val = np.min(data_nonzero)
        max_val = np.max(data_nonzero)

        all_means.append(mean_val)
        all_mins.append(min_val)
        all_maxs.append(max_val)

        print(f"{fname}: mean={mean_val:.2f}, min={min_val:.2f}, max={max_val:.2f}")

# Print general statistics
print("\n Overall summary across all images:")
print(f"Mean of means: {np.mean(all_means):.2f}")
print(f"Min of mins:   {np.min(all_mins):.2f}")
print(f"Max of maxs:   {np.max(all_maxs):.2f}")


cropped_1.img.nii.gz: mean=177.54, min=1.00, max=876.00
cropped_10.img.nii.gz: mean=228.36, min=1.00, max=1275.00
cropped_100.img.nii.gz: mean=231.17, min=1.00, max=1362.00
cropped_1000.img.nii.gz: mean=185.05, min=1.00, max=1323.00
cropped_101.img.nii.gz: mean=240.75, min=1.00, max=1509.00
cropped_102.img.nii.gz: mean=186.64, min=1.00, max=924.00
cropped_103.img.nii.gz: mean=240.83, min=1.00, max=1339.00
cropped_104.img.nii.gz: mean=189.66, min=1.00, max=945.00
cropped_105.img.nii.gz: mean=273.13, min=1.00, max=1876.00
cropped_106.img.nii.gz: mean=271.32, min=1.00, max=1202.00
cropped_107.img.nii.gz: mean=250.29, min=1.00, max=1352.00
cropped_108.img.nii.gz: mean=170.05, min=1.00, max=1006.00
cropped_109.img.nii.gz: mean=230.41, min=1.00, max=1495.00
cropped_11.img.nii.gz: mean=234.33, min=1.00, max=1311.00
cropped_110.img.nii.gz: mean=251.40, min=1.00, max=1513.00
cropped_111.img.nii.gz: mean=241.19, min=1.00, max=1491.00
cropped_112.img.nii.gz: mean=242.74, min=1.00, max=1354.00
cro